In [ ]:
%load_ext autoreload
%autoreload 2

from os import listdir
from os.path import isfile, join

import numpy as np
import matplotlib.pyplot as plt

import madmom
from madmom.features.onsets import SpectralOnsetProcessor, RNNOnsetProcessor, CNNOnsetProcessor

import sys
sys.path.append('../src')
from preprocessing import spectro_mini_db, spectro_mini_db_patches, concatenate_and_resample
from models import OLSPatchRegressor

na = np.newaxis

plt.rc('text', usetex=True)
plt.rc('font', family='serif')

In [ ]:
audio_dir = '../data/music_speech/music_wav'
audio_files = [f for f in listdir(audio_dir) if isfile(join(audio_dir, f))]

In [ ]:
from madmom.audio.filters import LogarithmicFilterbank
from madmom.features.beats import RNNBeatProcessor
from madmom.features.downbeats import RNNDownBeatProcessor, DBNDownBeatTrackingProcessor

rnn_beat_processor = RNNBeatProcessor(post_processor=None)
rnn_downbeat_processor = RNNDownBeatProcessor()

def get_rnn_beat_processor_features(signal):
    return np.array(rnn_beat_processor(signal)).T

def get_rnn_downbeat_features(signal):
    return rnn_downbeat_processor(signal)

processors = [proc() for proc in [SpectralOnsetProcessor, RNNOnsetProcessor, SpectralOnsetProcessor]]
processors.append(SpectralOnsetProcessor(onset_method='superflux', fps=200, filterbank=LogarithmicFilterbank, num_bands=24, log=np.log10))

processors +=  [get_rnn_beat_processor_features, get_rnn_downbeat_features] 

In [ ]:
ex_file = audio_files[0]
ex_path = join(audio_dir, ex_file)

audio = madmom.audio.Signal(ex_path)

out = [proc(ex_path) for proc in processors]

In [ ]:
out_con = concatenate_and_resample(out)

print(out_con.shape)